In [1]:
import torch
import pandas as pd
import pyaging as pya
import json
import os

In [2]:
#download PCClock Rdata file from https://yale.app.box.com/s/kq0b0a7lxckxjvaz7x5n4keaug7tewry

In [3]:
%%writefile pcclocks.r

library(dplyr)
library(tibble)
library(tidyr)
library(jsonlite)

load(file = "CalcAllPCClocks.RData")

print(ls(all.names = TRUE))

write_json(CalcPCHorvath1, "CalcPCHorvath1.json")
write_json(CpGs, "CpGs.json")

Writing pcclocks.r


In [4]:
os.system("Rscript pcclocks.r")
os.system("rm pcclocks.r")

0

In [5]:
with open('CpGs.json', 'r') as f:
    features = json.load(f)

with open('CalcPCHorvath1.json', 'r') as f:
    weights_dict = json.load(f)

model_weights = torch.tensor(weights_dict['model']).float()
intercept = torch.tensor(weights_dict['intercept']).float()
center = torch.tensor(weights_dict['center']).float()
rotation = torch.tensor(weights_dict['rotation']).float()

In [6]:
num_features = rotation.shape[0]
num_components = rotation.shape[1]

# Initialize the model
model = pya.models.PCLinearModel(input_dim=num_features, pc_dim=num_components)

model.center.data = center
model.rotation.data = rotation
model.linear.weight.data = model_weights.unsqueeze(0)
model.linear.bias.data = intercept

In [7]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': 'anti_log_linear',
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2022,
    'implementation_approved_by_author(s)': '⌛',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Higgins-Chen, Albert T., et al. \"A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking.\" Nature aging 2.7 (2022): 644-661.",
    'doi': "https://doi.org/10.1038/s43587-022-00248-2",
    "notes": None,
}

In [8]:
torch.save(weights_dict, '../weights/pchorvath2013.pt')
torch.save(metadata_dict, '../metadata/pchorvath2013.pt')

In [9]:
os.system("rm CalcPCHorvath1.json")
os.system("rm CpGs.json")

0